In [1]:
from __future__ import absolute_import, division, print_function

import os
import cv2
import tensorflow as tf
from tensorflow import keras
import numpy as np


dir_path = os.path.abspath('')
dataset_path =  os.path.join(dir_path,'Dataset')
one_path = os.path.join(dataset_path,'one')
two_path = os.path.join(dataset_path,'two')
three_path = os.path.join(dataset_path,'three')

test_labels = []
for i in range(0, 100):
    test_labels.append(0)
for i in range(0, 100):
    test_labels.append(1)
for i in range(0, 100):
    test_labels.append(2)
test_labels = np.array(test_labels)

one_path_test = os.path.join(dataset_path,'one','test')
two_path_test = os.path.join(dataset_path,'two','test')
three_path_test = os.path.join(dataset_path,'three','test')
test_images = []

for i in range(0, 100):
    image = cv2.imread(os.path.join(one_path_test, "one" + str(i) + '.png'))
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    test_images.append(gray_image)
for i in range(0, 100):
    image = cv2.imread(os.path.join(two_path_test, "two" + str(i) + '.png'))
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    test_images.append(gray_image)
for i in range(0, 100):
    image = cv2.imread(os.path.join(three_path_test, "three" + str(i) + '.png'))
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    test_images.append(gray_image)
test_images = np.array(test_images)
test_images =testest_images = test_images/255

new_model = keras.models.load_model('my_model.h5')
new_model.summary()


W0622 11:08:56.878140 10064 deprecation.py:506] From c:\users\owner\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\ops\init_ops.py:97: calling GlorotUniform.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0622 11:08:56.882141 10064 deprecation.py:506] From c:\users\owner\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\ops\init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0622 11:08:56.886140 10064 deprecation.py:506] From c:\users\owner\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\ops\init_ops.py:97: c

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 8900)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 128)               1139328   
_________________________________________________________________
dense_3 (Dense)              (None, 3)                 387       
Total params: 1,139,715
Trainable params: 1,139,715
Non-trainable params: 0
_________________________________________________________________


In [2]:

loss, acc = new_model.evaluate(test_images, test_labels)

print("Restored model, accuracy: {:5.2f}%".format(100*acc))


predictions = new_model.predict(test_images)


300/300 [==============================] - 0s 373us/sample - loss: 0.3766 - acc: 0.9000
Restored model, accuracy: 90.00%


In [3]:
predictions[9]

array([9.9989510e-01, 1.0494961e-04, 6.1501448e-10], dtype=float32)

In [4]:
np.argmax(predictions[9])

0

In [5]:
# organize imports
import cv2
import imutils
import numpy as np
from PIL import Image


# global variables
bg = None


def run_avg(image, aWeight):
    global bg
    # initialize the background
    if bg is None:
        bg = image.copy().astype("float")
        return

    # compute weighted average, accumulate it and update the background
    cv2.accumulateWeighted(image, bg, aWeight)


def segment(image, threshold=25):
    global bg
    # find the absolute difference between background and current frame
    diff = cv2.absdiff(bg.astype("uint8"), image)

    # threshold the diff image so that we get the foreground
    thresholded = cv2.threshold(diff,
                                threshold,
                                255,
                                cv2.THRESH_BINARY)[1]

    # get the contours in the thresholded image
    ( cnts, _) = cv2.findContours(thresholded.copy(),
                                    cv2.RETR_EXTERNAL,
                                    cv2.CHAIN_APPROX_SIMPLE)

    # return None, if no contours detected
    if len(cnts) == 0:
        return
    else:
        # based on contour area, get the maximum contour which is the hand
        segmented = max(cnts, key=cv2.contourArea)
        return (thresholded, segmented)


def main():
    # initialize weight for running average
    aWeight = 0.5

    # get the reference to the webcam
    camera = cv2.VideoCapture(0)

    # region of interest (ROI) coordinates
    top, right, bottom, left = 10, 350, 225, 590

    # initialize num of frames
    num_frames = 0
    image_num = 0

    start_recording = False
   # sample_name = 'two'

    # keep looping, until interrupted
    while (True):
        # get the current frame
        (grabbed, frame) = camera.read()
        if (grabbed == True):

            # resize the frame
            frame = imutils.resize(frame, width=700)

            # flip the frame so that it is not the mirror view
            frame = cv2.flip(frame, 1)

            # clone the frame
            clone = frame.copy()

            # get the height and width of the frame
            (height, width) = frame.shape[:2]

            # get the ROI
            roi = frame[top:bottom, right:left]

            # convert the roi to grayscale and blur it
            gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
            gray = cv2.GaussianBlur(gray, (7, 7), 0)

            # to get the background, keep looking till a threshold is reached
            # so that our running average model gets calibrated
            if num_frames < 30:
                run_avg(gray, aWeight)
                print(num_frames)
            else:
                # segment the hand region
                hand = segment(gray)

                # check whether hand region is segmented
                if hand is not None:
                    # if yes, unpack the thresholded image and
                    # segmented region
                    (thresholded, segmented) = hand
                    cv2.imwrite( 'predict.png', thresholded)
                    basewidth = 100
                    img = Image.open('predict.png')
                    wpercent = (basewidth / float(img.size[0]))
                    hsize = int((float(img.size[1]) * float(wpercent)))
                    img = img.resize((basewidth, hsize), Image.ANTIALIAS)
                    img.save('predict.png')
                    image = cv2.imread('predict.png')
                    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
                    gray_image = gray_image/255
                    gray_image = np.reshape(gray_image,[1,89,100])
                    prediction = new_model.predict(gray_image)
                    print(prediction)
                    print(np.argmax(prediction))
                    print('hand')
                    # draw the segmented region and display the frame
                    cv2.drawContours(clone, [segmented + (right, top)], -1, (0, 0, 255))
                    if start_recording:
                        # Mention the directory in which you wanna store the images followed by the image name
                        cv2.imwrite("Dataset/" + sample_name +'/test' + '/' + sample_name  + str(image_num) + '.png', thresholded)
                        image_num += 1
                    cv2.imshow("Thesholded", thresholded)

            # draw the segmented hand
            cv2.rectangle(clone, (left, top), (right, bottom), (0, 255, 0), 2)

            # increment the number of frames
            num_frames += 1

            # display the frame with segmented hand
            cv2.imshow("Video Feed", clone)

            # observe the keypress by the user
            keypress = cv2.waitKey(1) & 0xFF

            # if the user pressed "q", then stop looping
            if keypress == ord("q") or image_num > 100:
                break

            if keypress == ord("s"):
                start_recording = True

        else:
            print("[Warning!] Error input, Please check your(camra Or video)")
            break

    # free up memory
    camera.release()
    cv2.destroyAllWindows()


In [6]:
main()

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
[[0.33254138 0.35695967 0.31049898]]
1
hand
[[0.33254138 0.35695967 0.31049898]]
1
hand
[[0.24624345 0.588524   0.16523254]]
1
hand
[[0.24624345 0.588524   0.16523254]]
1
hand
[[0.3731651 0.3006121 0.3262228]]
0
hand
[[0.3731651 0.3006121 0.3262228]]
0
hand
[[0.35937285 0.29356954 0.34705755]]
0
hand
[[0.35937285 0.29356954 0.34705755]]
0
hand
[[0.3583631  0.29268694 0.34894994]]
0
hand
[[0.3583631  0.29268694 0.34894994]]
0
hand
[[0.3616828  0.2844073  0.35390994]]
0
hand
[[0.3616828  0.2844073  0.35390994]]
0
hand
[[0.3614091  0.27493823 0.3636527 ]]
2
hand
[[0.3614091  0.27493823 0.3636527 ]]
2
hand
[[0.35915783 0.29396057 0.3468816 ]]
0
hand
[[0.35915783 0.29396057 0.3468816 ]]
0
hand
[[0.3614091  0.27493823 0.3636527 ]]
2
hand
[[0.3614091  0.27493823 0.3636527 ]]
2
hand
[[0.36409926 0.26904333 0.36685735]]
2
hand
[[0.36409926 0.26904333 0.36685735]]
2
hand
[[0.3599374  0.282454   0.35760865]]
0
hand
[[

[[9.9945599e-01 5.4406543e-04 1.8806432e-08]]
0
hand
[[9.9945599e-01 5.4406543e-04 1.8806432e-08]]
0
hand
[[9.9984658e-01 1.5340316e-04 6.6177508e-09]]
0
hand
[[9.9984658e-01 1.5340316e-04 6.6177508e-09]]
0
hand
[[9.9987817e-01 1.2179945e-04 5.1743729e-09]]
0
hand
[[9.9987817e-01 1.2179945e-04 5.1743729e-09]]
0
hand
[[9.9982280e-01 1.7721191e-04 8.2019929e-09]]
0
hand
[[9.9982280e-01 1.7721191e-04 8.2019929e-09]]
0
hand
[[9.9992132e-01 7.8649551e-05 5.2272613e-09]]
0
hand
[[9.9992132e-01 7.8649551e-05 5.2272613e-09]]
0
hand
[[9.9994576e-01 5.4264783e-05 4.9504574e-09]]
0
hand
[[9.9994576e-01 5.4264783e-05 4.9504574e-09]]
0
hand
[[9.999337e-01 6.624864e-05 6.814429e-09]]
0
hand
[[9.999337e-01 6.624864e-05 6.814429e-09]]
0
hand
[[9.9994612e-01 5.3823926e-05 5.4073137e-09]]
0
hand
[[9.9994612e-01 5.3823926e-05 5.4073137e-09]]
0
hand
[[9.9995875e-01 4.1185638e-05 4.3638981e-09]]
0
hand
[[9.9995875e-01 4.1185638e-05 4.3638981e-09]]
0
hand
[[9.9996984e-01 3.0164536e-05 2.5187841e-09]]
0
hand

[[9.99928355e-01 7.16957220e-05 1.31074565e-08]]
0
hand
[[9.9993026e-01 6.9752001e-05 2.8476718e-08]]
0
hand
[[9.9993026e-01 6.9752001e-05 2.8476718e-08]]
0
hand
[[9.9993765e-01 6.2301238e-05 3.3142932e-08]]
0
hand
[[9.9993765e-01 6.2301238e-05 3.3142932e-08]]
0
hand
[[9.9984682e-01 1.5311308e-04 5.0478114e-08]]
0
hand
[[9.9984682e-01 1.5311308e-04 5.0478114e-08]]
0
hand
[[9.999161e-01 8.393399e-05 3.593288e-08]]
0
hand
[[9.999161e-01 8.393399e-05 3.593288e-08]]
0
hand
[[9.9992716e-01 7.2859068e-05 3.0049389e-08]]
0
hand
[[9.9992716e-01 7.2859068e-05 3.0049389e-08]]
0
hand
[[9.9989307e-01 1.0697108e-04 3.2631693e-08]]
0
hand
[[9.9989307e-01 1.0697108e-04 3.2631693e-08]]
0
hand
[[9.999114e-01 8.855328e-05 3.440194e-08]]
0
hand
[[9.999114e-01 8.855328e-05 3.440194e-08]]
0
hand
[[9.9985147e-01 1.4846682e-04 2.4147175e-08]]
0
hand
[[9.9985147e-01 1.4846682e-04 2.4147175e-08]]
0
hand
[[9.9988878e-01 1.1115305e-04 1.9457707e-08]]
0
hand
[[9.9988878e-01 1.1115305e-04 1.9457707e-08]]
0
hand
[[

0
hand
[[9.9965227e-01 3.4709679e-04 5.9618338e-07]]
0
hand
[[9.9969268e-01 3.0681922e-04 5.0574721e-07]]
0
hand
[[9.9969268e-01 3.0681922e-04 5.0574721e-07]]
0
hand
[[9.9972063e-01 2.7873626e-04 6.2173888e-07]]
0
hand
[[9.9972063e-01 2.7873626e-04 6.2173888e-07]]
0
hand
[[9.9966228e-01 3.3675047e-04 9.7262091e-07]]
0
hand
[[9.9966228e-01 3.3675047e-04 9.7262091e-07]]
0
hand
[[9.9965215e-01 3.4699682e-04 8.8550996e-07]]
0
hand
[[9.9965215e-01 3.4699682e-04 8.8550996e-07]]
0
hand
[[9.9956137e-01 4.3816626e-04 4.3812048e-07]]
0
hand
[[9.9956137e-01 4.3816626e-04 4.3812048e-07]]
0
hand
[[9.9959606e-01 4.0334789e-04 5.9717695e-07]]
0
hand
[[9.9959606e-01 4.0334789e-04 5.9717695e-07]]
0
hand
[[9.9968517e-01 3.1404677e-04 8.6018036e-07]]
0
hand
[[9.9968517e-01 3.1404677e-04 8.6018036e-07]]
0
hand
[[9.9954826e-01 4.5066510e-04 1.1103574e-06]]
0
hand
[[9.9954826e-01 4.5066510e-04 1.1103574e-06]]
0
hand
[[9.9965882e-01 3.4015940e-04 1.0653628e-06]]
0
hand
[[9.9965882e-01 3.4015940e-04 1.0653628

[[9.9859065e-01 1.3825292e-03 2.6746753e-05]]
0
hand
[[9.9859065e-01 1.3825292e-03 2.6746753e-05]]
0
hand
[[9.9843627e-01 1.5266141e-03 3.7112521e-05]]
0
hand
[[9.9843627e-01 1.5266141e-03 3.7112521e-05]]
0
hand
[[9.9860567e-01 1.3090039e-03 8.5331398e-05]]
0
hand
[[9.9860567e-01 1.3090039e-03 8.5331398e-05]]
0
hand
[[0.9762403  0.02131809 0.00244171]]
0
hand
[[0.9762403  0.02131809 0.00244171]]
0
hand
[[0.28672254 0.5532894  0.159988  ]]
1
hand
[[0.28672254 0.5532894  0.159988  ]]
1
hand
[[0.3094775  0.5629623  0.12756021]]
1
hand
[[0.3094775  0.5629623  0.12756021]]
1
hand
[[0.29638377 0.52468276 0.17893353]]
1
hand
[[0.29638377 0.52468276 0.17893353]]
1
hand
[[0.2912938  0.508365   0.20034117]]
1
hand
[[0.2912938  0.508365   0.20034117]]
1
hand
[[0.34384587 0.544299   0.11185507]]
1
hand
[[0.34384587 0.544299   0.11185507]]
1
hand
[[0.298475   0.5312153  0.17030968]]
1
hand
[[0.298475   0.5312153  0.17030968]]
1
hand
[[0.31773475 0.4025214  0.27974385]]
1
hand
[[0.31773475 0.4025214

[[0.17386228 0.5526212  0.27351657]]
1
hand
[[0.17386228 0.5526212  0.27351657]]
1
hand
[[0.17256492 0.56452084 0.26291427]]
1
hand
[[0.17256492 0.56452084 0.26291427]]
1
hand
[[0.14090495 0.64958495 0.20951012]]
1
hand
[[0.14090495 0.64958495 0.20951012]]
1
hand
[[0.3933511  0.30195034 0.3046986 ]]
0
hand
[[0.3933511  0.30195034 0.3046986 ]]
0
hand
[[0.30524296 0.3269269  0.36783016]]
2
hand
[[0.30524296 0.3269269  0.36783016]]
2
hand
[[0.19729455 0.5045039  0.2982016 ]]
1
hand
[[0.19729455 0.5045039  0.2982016 ]]
1
hand
[[0.28603083 0.3365011  0.37746814]]
2
hand
[[0.28603083 0.3365011  0.37746814]]
2
hand
[[0.3787541  0.32303655 0.29820937]]
0
hand
[[0.3787541  0.32303655 0.29820937]]
0
hand
[[0.31608894 0.3302702  0.35364088]]
2
hand
[[0.31608894 0.3302702  0.35364088]]
2
hand
[[0.3140308  0.33932537 0.3466438 ]]
2
hand
[[0.3140308  0.33932537 0.3466438 ]]
2
hand
[[0.2033182  0.44238093 0.35430092]]
1
hand
[[0.2033182  0.44238093 0.35430092]]
1
hand
[[0.14759961 0.5911066  0.261293

hand
[[0.31442842 0.32182753 0.36374402]]
2
hand
[[0.31442842 0.32182753 0.36374402]]
2
hand
[[0.28413972 0.32353824 0.392322  ]]
2
hand
[[0.28413972 0.32353824 0.392322  ]]
2
hand
[[0.31381083 0.33261043 0.35357872]]
2
hand
[[0.31381083 0.33261043 0.35357872]]
2
hand
[[0.32222912 0.3243718  0.3533991 ]]
2
hand
[[0.32222912 0.3243718  0.3533991 ]]
2
hand
[[0.31083578 0.33706442 0.35209984]]
2
hand
[[0.31083578 0.33706442 0.35209984]]
2
hand
[[0.31494024 0.34891686 0.33614287]]
1
hand
[[0.31494024 0.34891686 0.33614287]]
1
hand
[[0.30944288 0.35025162 0.34030548]]
1
hand
[[0.30944288 0.35025162 0.34030548]]
1
hand
[[0.31960994 0.32432103 0.35606906]]
2
hand
[[0.31960994 0.32432103 0.35606906]]
2
hand
[[0.30872667 0.36239934 0.32887408]]
1
hand
[[0.30872667 0.36239934 0.32887408]]
1
hand
[[0.27490544 0.36033893 0.3647557 ]]
2
hand
[[0.27490544 0.36033893 0.3647557 ]]
2
hand
[[0.3411529  0.32945022 0.32939684]]
0
hand
[[0.3411529  0.32945022 0.32939684]]
0
hand
[[0.30576983 0.30820736 0.3

[[0.21530966 0.40376326 0.38092712]]
1
hand
[[0.21530966 0.40376326 0.38092712]]
1
hand
[[0.1583478  0.54209363 0.29955858]]
1
hand
[[0.1583478  0.54209363 0.29955858]]
1
hand
[[0.19336705 0.44043624 0.36619663]]
1
hand
[[0.19336705 0.44043624 0.36619663]]
1
hand
[[0.3267365  0.34540224 0.32786128]]
1
hand
[[0.3267365  0.34540224 0.32786128]]
1
hand
[[0.36488765 0.28423718 0.35087523]]
0
hand
[[0.36488765 0.28423718 0.35087523]]
0
hand
[[0.35960177 0.30478168 0.33561662]]
0
hand
[[0.35960177 0.30478168 0.33561662]]
0
hand
[[0.3674675  0.2773262  0.35520634]]
0
hand
[[0.37881532 0.2773333  0.34385136]]
0
hand
[[0.37881532 0.2773333  0.34385136]]
0
hand
[[0.3645168  0.28133214 0.3541511 ]]
0
hand
[[0.3645168  0.28133214 0.3541511 ]]
0
hand
[[0.36264735 0.28417602 0.35317662]]
0
hand
[[0.36264735 0.28417602 0.35317662]]
0
hand
[[0.35832208 0.2932608  0.34841716]]
0
hand
[[0.35832208 0.2932608  0.34841716]]
0
hand
[[0.31638202 0.3843472  0.29927087]]
1
hand
[[0.31638202 0.3843472  0.299270